# scratch work

In [1]:
import jax

jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp

from astropy.time import Time
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.jplhorizons import Horizons

from jorbit import Particle

In [2]:
t0 = Time("2024-12-01 00:00")
t1 = Time("2025-12-01 00:00")

obj = Horizons(id="274301", location="@0", epochs=[t0.tdb.jd, t1.tdb.jd])
vecs = obj.vectors(refplane="earth")

x0 = jnp.array([vecs["x"][0], vecs["y"][0], vecs["z"][0]])
v0 = jnp.array([vecs["vx"][0], vecs["vy"][0], vecs["vz"][0]])

In [3]:
p = Particle(x=x0, v=v0, log_gm=-jnp.inf, time=t1)
p

yay
<class 'jax.tree_util.Partial'>
<class 'jax.tree_util.Partial'>


Particle: unnamed

In [4]:
p.gravity(1.0)

AttributeError: 'float' object has no attribute 'time'